Azure ML & Azure Databricks notebooks by Parashar Shah.

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

#Data Ingestion

In [3]:
import os
import urllib

In [4]:
# Download AdultCensusIncome.csv from Azure CDN. This file has 32,561 rows.
basedataurl = "https://amldockerdatasets.azureedge.net"
datafile = "AdultCensusIncome.csv"
datafile_dbfs = os.path.join("/dbfs", datafile)

if os.path.isfile(datafile_dbfs):
    print("found {} at {}".format(datafile, datafile_dbfs))
else:
    print("downloading {} to {}".format(datafile, datafile_dbfs))
    urllib.request.urlretrieve(os.path.join(basedataurl, datafile), datafile_dbfs)

found AdultCensusIncome.csv at /dbfs/AdultCensusIncome.csv

In [5]:
# Create a Spark dataframe out of the csv file.
data_all = sqlContext.read.format('csv').options(header='true', inferSchema='true', ignoreLeadingWhiteSpace='true', ignoreTrailingWhiteSpace='true').load(datafile)
print("({}, {})".format(data_all.count(), len(data_all.columns)))
data_all.printSchema()

(32561, 15)
root
-- age: integer (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: integer (nullable = true)
-- education: string (nullable = true)
-- education-num: integer (nullable = true)
-- marital-status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- sex: string (nullable = true)
-- capital-gain: integer (nullable = true)
-- capital-loss: integer (nullable = true)
-- hours-per-week: integer (nullable = true)
-- native-country: string (nullable = true)
-- income: string (nullable = true)

In [6]:
#renaming columns
columns_new = [col.replace("-", "_") for col in data_all.columns]
data_all = data_all.toDF(*columns_new)
data_all.printSchema()

root
-- age: integer (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: integer (nullable = true)
-- education: string (nullable = true)
-- education_num: integer (nullable = true)
-- marital_status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- sex: string (nullable = true)
-- capital_gain: integer (nullable = true)
-- capital_loss: integer (nullable = true)
-- hours_per_week: integer (nullable = true)
-- native_country: string (nullable = true)
-- income: string (nullable = true)

In [7]:
#ensure that you see a table with 5 rows and various columns
display(data_all.limit(5))

age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


#Data Preparation

In [9]:
# Choose feature columns and the label column.
label = "income"
xvars = set(data_all.columns) - {label}

print("label = {}".format(label))
print("features = {}".format(xvars))

data = data_all.select([*xvars, label])

# Split data into train and test.
train, test = data.randomSplit([0.80, 0.20], seed=13)

print("train ({}, {})".format(train.count(), len(train.columns)))
print("test ({}, {})".format(test.count(), len(test.columns)))

label = income
features = {'race', 'capital_gain', 'workclass', 'hours_per_week', 'native_country', 'sex', 'education', 'age', 'education_num', 'marital_status', 'capital_loss', 'occupation', 'fnlwgt', 'relationship'}
train (25967, 15)
test (6594, 15)

#Data Persistence

In [11]:
# Write the train and test data sets to intermediate storage
train_data_path = "AdultCensusIncomeTrain"
test_data_path = "AdultCensusIncomeTest"

train_data_path_dbfs = os.path.join("/dbfs", "AdultCensusIncomeTrain")
test_data_path_dbfs = os.path.join("/dbfs", "AdultCensusIncomeTest")

train.write.mode('overwrite').parquet(train_data_path)
test.write.mode('overwrite').parquet(test_data_path)
print("train and test datasets saved to {} and {}".format(train_data_path_dbfs, test_data_path_dbfs))

train and test datasets saved to /dbfs/AdultCensusIncomeTrain and /dbfs/AdultCensusIncomeTest